In [2]:
from paddleocr import PaddleOCR
from PIL import Image
import torch
from huggingface_hub import hf_hub_download
from transformers import DetrFeatureExtractor
from transformers import TableTransformerForObjectDetection
import numpy as np
from matplotlib import pyplot as plt
import cv2
from threading import Thread
from threading import RLock
ocr_model = PaddleOCR(lang='en',use_angle_cls=False,show_log=False)

In [3]:
def scan(img,orig_img):
    # Repeated Closing operation to remove text from the document.
    kernel = np.ones((5, 5), np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=3)
    canny = cv2.Canny(img, 70, 300)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    
    # Finding contours for the detected edges.
    contours, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Keeping only the largest detected contour.
    page = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
 
    # Detecting Edges through Contour approximation.
    # Loop over the contours.
    if len(page) == 0:
        return orig_img
    for c in page:
        # Approximate the contour.
        epsilon = 0.02 * cv2.arcLength(c, True)
        corners = cv2.approxPolyDP(c, epsilon, True)
        # If our approximated contour has four points.
        if len(corners) == 4:
            break
    # Sorting the corners and converting them to desired shape.
    corners = sorted(np.concatenate(corners).tolist())
    # For 4 corner points being detected.
    corners = order_points(corners)
 
    destination_corners = find_dest(corners)
 
    h, w = orig_img.shape[:2]
    # Getting the homography.
    M = cv2.getPerspectiveTransform(np.float32(corners), np.float32(destination_corners))
    # Perspective transform using homography.
    final = cv2.warpPerspective(orig_img, M, (destination_corners[2][0], destination_corners[2][1]),flags=cv2.INTER_LINEAR)
    return Image.fromarray(final)

In [4]:
def order_points(pts):
    '''Rearrange coordinates to order:
      top-left, top-right, bottom-right, bottom-left'''
    rect = np.zeros((4, 2), dtype='float32')
    pts = np.array(pts)
    s = pts.sum(axis=1)
    # Top-left point will have the smallest sum.
    rect[0] = pts[np.argmin(s)]
    # Bottom-right point will have the largest sum.
    rect[2] = pts[np.argmax(s)]
 
    diff = np.diff(pts, axis=1)
    # Top-right point will have the smallest difference.
    rect[1] = pts[np.argmin(diff)]
    # Bottom-left will have the largest difference.
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect.astype('int').tolist()

In [5]:
def find_dest(pts):
    (tl, tr, br, bl) = pts
    # Finding the maximum width.
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
 
    # Finding the maximum height.
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # Final destination co-ordinates.
    destination_corners = [[0, 0], [maxWidth, 0], [maxWidth, maxHeight], [0, maxHeight]]
 
    return order_points(destination_corners)

In [6]:
def scaler(box,xscale,yscale,xmax,ymax):
    box[0]*=1-xscale
    box[1]*=1-yscale
    box[2]*=1+xscale
    box[3]*=1+yscale
    if box[2]>xmax:
        box[2] = xmax
    if box[3]>ymax:
        box[3] = ymax
    return box

In [7]:
def table_detection(image):
    width, height = image.size
    feature_extractor = DetrFeatureExtractor()
    encoding = feature_extractor(image, return_tensors="pt")
    model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

    with torch.no_grad():
        outputs = model(**encoding)

    results = feature_extractor.post_process_object_detection(outputs, threshold=0.7, target_sizes=[(height, width)])[0]

    cropped_img = []
    scale = 0.1
    all_boxes = []
    for i in range(len(results['scores'])):
        bounding_box = scaler(results['boxes'][i].tolist(),scale,scale,width,height)
        all_boxes.append(bounding_box)
        cropped_img.append(image.crop(bounding_box))
    
    for i in all_boxes:
        mask_height = int(i[3] - i[1])
        mask_width = int(i[2] - i[0])
        square = np.full((mask_height, mask_width),255)
        square_img = Image.fromarray(square.astype(np.uint8))
        image.paste(square_img,(int(i[0]),int(i[1])))
        
    return cropped_img,image

In [8]:
def table_structure_detection(image):
    width, height = image.size
    feature_extractor = DetrFeatureExtractor()
    encoding = feature_extractor(image, return_tensors="pt")
    model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-structure-recognition")
    
    with torch.no_grad():
        outputs = model(**encoding)
    target_sizes = [image.size[::-1]]
    results = feature_extractor.post_process_object_detection(outputs, threshold=0.7, target_sizes=target_sizes)[0]
    
    output = {"headers":[],"row_data":[]}
    y_scale = 0.03
    
    #labels == 1 is col_data, 2 is row_data, 3 is col_header_data, 4 is row_header_data
    threads = []
    result = ThreadSafeList()
    for i in range(len(results['boxes'])):
        if results['labels'][i] == 4:
            bounding_box = scaler(results['boxes'][i].tolist(),1,y_scale,width,height)
            row_header_img = image.crop(bounding_box)
            np_img = np.asarray(row_header_img)
            result_header = ocr_model.ocr(np_img)
            for i in result_header[0]:
                output["headers"].append(i[1][0])
        elif results['labels'][i] == 2:
            bounding_box = scaler(results['boxes'][i].tolist(),1,y_scale,width,height)
            row_img = image.crop(bounding_box)
            np_img = np.asarray(row_img)
            threads.append(Thread(target=ocr_thread, args=(np_img,result)))
    
    for i in threads:
        i.start()
    for i in threads:
        i.join()
        
    for i in range(len(result)):
        row_entry = []
        for j in result[i][0]:
            row_entry.append(j[1][0])
        output["row_data"].append(row_entry)
    return output

In [9]:
class ThreadSafeList:
    def __init__(self):
        self._list = list()
        self._lock = RLock()
    def append(self,data):
        with self._lock:
            self._list.append(data)
    def __getitem__(self, key):
        return self._list[key]
    def __len__(self):
        return len(self._list)

In [10]:
def get_midpoints(data):
    output = {}
    for i in data:
        x_pos = sum([j[0] for j in i[0]])/4.0
        y_pos = sum([j[1] for j in i[0]])/4.0
        output[(x_pos,y_pos)] = i[1][0]
    return output

In [11]:
def organize_table(result):
    horizontal_org = {}
    horizontal_unorg = {}
    vertical_org = {}
    vertical_unorg = {}
    
    midpoints = get_midpoints(result[0])

    for i in result[0]:
        hori_ls = []
        verti_ls = []
        if not any(j in i[1][0] for j in '1234567890'):
            for k,v in midpoints.items():
                max_x = max([n[0] for n in i[0]])
                max_y = max([n[1] for n in i[0]])
                min_x = min([n[0] for n in i[0]])
                min_y = min([n[1] for n in i[0]])

                if min_y<k[1]<max_y and v != i[1][0]:
                    hori_ls.append((k[0]-i[0][0][0],v))
                if min_x<k[0]<max_x and v != i[1][0]:
                    verti_ls.append((k[1]-i[0][0][1],v))
        if len(hori_ls):
            min_val = min([n[0] if n[0]>0 else 100000 for n in hori_ls])
            for j in hori_ls:
                if j[0] == min_val and min_val>0:
                    if any(k in j[1] for k in '1234567890'):
                        horizontal_org[i[1][0]] = j[1]
                    horizontal_unorg[i[1][0]] = j[1]

        if len(verti_ls):
            min_val = min([n[0] if n[0]>0 else 100000 for n in verti_ls])
            for j in verti_ls:
                if j[0] == min_val and min_val>0:
                    if any(k in j[1] for k in '1234567890'):
                        vertical_org[i[1][0]] = j[1]
                    vertical_unorg[i[1][0]] = j[1]

    if len(vertical_org)>len(horizontal_org):
        return vertical_unorg
    else:
        return horizontal_unorg

In [12]:
def run_ocr_no_temp(image,edge_detect=False):
    if edge_detect:
        #Peform edge detection
        img = cv2.cvtColor(np.array(image), cv2.IMREAD_GRAYSCALE)
        orig_img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        image = scan(img,orig_img)
        
    #Search for table
    table,image_no_table = table_detection(image)
    
    #Extract unorganized data
    unorganized_data = np.asarray(image_no_table)
    result = ocr_model.ocr(unorganized_data)
    extracted_data = organize_table(result)
    extracted_data["Table_Data"] = {"headers":[],"row_data":[],}
    
    #Extract Table Data
    table_output = []
    for i in table:
        table_output.append(table_structure_detection(i))
    
    #Get possible header info
    for n in range(len(table_output)):
        if len(table_output[n]["headers"]) == 0:
            for i in table_output[n]["row_data"]:
                data_str = (",").join(i)
                if not any(j in data_str for j in '1234567890'):
                    table_output[n]["headers"].append(i)
                    table_output[n]["row_data"].remove(i)

    #Join Data
    for n in range(len(table_output)):
        extracted_data["Table_Data"]["headers"].append(table_output[n]["headers"])
        extracted_data["Table_Data"]["row_data"].append(table_output[n]["row_data"])
    
    return extracted_data

In [20]:
img_path = "../data/Invoice/CamScanner 06-20-2023 11.20_115.jpg"
data = run_ocr_no_temp(img_path,edge_detect=False)

[2023/07/17 06:43:59] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/07/17 06:44:32] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/07/17 06:44:32] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/07/17 06:44:32] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/07/17 06:44:32] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/07/17 06:44:32] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/07/17 06:44:32] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier w

Exception in thread Thread-61 (ocr_thread):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2637/370562754.py", line 2, in ocr_thread
Exception in thread Thread-71 (ocr_thread):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
  File "/opt/conda/lib/python3.10/site-packages/paddleocr/paddleocr.py", line 555, in ocr
        self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
dt_boxes, rec_res, _ = self.__call__(img, cls)
  File "/opt/conda/lib/python3.10/site-packages/paddleocr/tools/infer/predict_system.py", line 95, in __call__
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2637/370562754.py", line 2, in ocr_thread
    rec_res, elapse = self.text_recognizer(img

In [21]:
data

{'GUO RUI CHENG PTE LTD': 'Co.REG. NO: 201528808M GST REG. NO: 201528808M',
 'EMAIL: grctrading@singnet.com.sg': 'TEL: 6779 5738 FAX: 6779 5581',
 'REVERSHINEV': 'ow',
 'Goi': 'All',
 'All': 'Re',
 'ow': "CUSTOMER'S SIGNATURE & CHOP",
 'Table_Data': {'headers': [[['UNIT PRICE', 'AMOUNT', 'PARTICULABS']]],
  'row_data': [[['BLOCK 103 YISHUN RING ROAD', 'z1# DO No'],
    ['KIM ENG MINI SUPERMARKET', 'Invoice No#', ':15/034573'],
    ['1EL:6/53660817', 'Page'],
    ['SINGAPORE 760703',
     '1t #k FR 3',
     'Terms',
     '30 Days',
     '..',
     'TEL : 67536888 FAX: 67578108'],
    ['3s91KmIm CX', 'Date', ':27/05/2020'],
    ['GUAINTBY',
     '$32.70',
     '$32.70',
     '2P GULA MELACCA 500G X 20',
     '2P # 500G X 20 (3)'],
    ['MAUNGGREEN PEAS 397G',
     '0.0',
     'MIUI 190 LUNCHEON MEAT',
     '$15.00',
     '$30.00',
     '190G'],
    ['L', '$7.00', '$7.00', 'DRIED DRAWING', '422.00'],
    ['L', '$7.00', '$7.00', 'ASAM KEPING', 'EI', 'DRIED DRAWING', '422.00'],
    ['24X35G